In [69]:
from doctest import FAIL_FAST
import pandas as pd
from os import path
from io import open
import unicodedata
import string
import re
import random
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import warnings

warnings.filterwarnings("ignore")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class CreateDataset():
    def read_csv(self, path):
        df = pd.read_csv(path, sep='\t', header=None)
        df = df[[0,1]]
        return df

# Определим по умолчанию 2 токена которые будут нам информировать о начале предложения и конце предложения (SOS и EOS):
SOS_token = 0
EOS_token = 1

# Создадим объект словаря нашего языка, который будет хранить данные по маппингу слов - index2word и обратно word2index и плюс второстепенные методы по добавлению токена и обработке предложений:
class LanguageVocabulary(object):
    def __init__(self, name):
        # название языка
        self.name = name
        # словарик word2index который хранит соответственно кодировку слова в целочисленный индекс словаря
        self.word2index = {}
        # обычный словарик который хранит распределение слов, сколько слов мы использовали и сколько обнаружили
        self.word2count = {}
        # Обратный словарик словарю word2index где хранятся уже индексы и замаппенные слова к каждому индексу, нужен будет для расшифровки последовательности
        self.index2word = {0: "SOS", 1: "EOS"}
        # Count SOS and EOS, храним просто общее количество слов в нашем словаре, то есть количество токенов в сформированном словарике нашего языка
        self.n_words = 2

    def add_sentence(self, sentence):
        """
        Метод класса, для добавления предложения в словарь.
        Каждое предложение поступающее к нам, будет разбираться на
        примитивные токены и добавляться в словарь при помощи метода класса addword()
        """
        for word in sentence.split(' '):
            self.add_word(word)


    def add_word(self, word):
        # проверяем не входит ли наше слово в словарь word2index
        if word not in self.word2index:
            # добавляем в качестве ключа слово а в качестве значения последнее n_words
            self.word2index[word] = self.n_words
            # меняем на единичку
            self.word2count[word] = 1
            # и соответственно меняем и index2word словарик добавляя уже слово для декодирования
            self.index2word[self.n_words] = word
            # инкрементируем n_words
            self.n_words += 1
        else:
            # Если такое уже слово есть просто добавляем 1 что добавилось одно слово
            self.word2count[word] += 1


            # Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicode_to_ascii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalize_string(s):
    # Декодируем из юникода в ascii
    s = unicode_to_ascii(s.lower().strip())
    # Что означает данное регулярное выражение - точку, !, ? меняем на пробел чтобы этот символ стоял отдельно от всех
    # https://docs.python.org/3/library/re.html - стандартная (родная) библиотка Python которая нужна для работы с регулярными выражениями
    s = re.sub(r"([.!?])", r" \1", s)
    # оставляем только наборы символов указанных в паттерне регулярного выражения остальное заменим на пробел
    s = re.sub(r"[^a-zA-Zа-яА-ЯёЁĄąĆćĘęŁłŃńÓóŚśŹźŻżÄäÖöÜü\u4e00-\u9fff.!?]+", r" ", s)
   
    return s


def read_languages(lang1, lang2, reverse=False):
    print("Reading lines...")
    # Берем документ корпуса, лежащий в директории ./data/___.txt подставляя значения указанных языков в нашем случае eng-fra, он читается бьется на предложения
    #lines = open('/home/mikhail/it-academy/модуль 5/lecture_7_deepRNN/data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').read().strip().split('\n')
    lines = [l[0]+'\t'+l[1] for l in df.values]
    #print(lines[:10])
    # Разбиваем построчно и нормализуем строку:
    pairs = [[normalize_string(s) for s in l.split('\t')] for l in lines]
    # Можем создавать и проходить как с целевого языка на исходный так и наоборот:
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = LanguageVocabulary(lang2)
        output_lang = LanguageVocabulary(lang1)
    else:
        input_lang = LanguageVocabulary(lang1)
        output_lang = LanguageVocabulary(lang2)
    return input_lang, output_lang, pairs

MAX_LENGTH = 10

"""
eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filter_pair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH and p[1].startswith(eng_prefixes)

def filter_pairs(pairs):
    return [pair for pair in pairs if filter_pair(pair)]
"""

def prepare_data(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = read_languages(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    #pairs = filter_pairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.add_sentence(pair[0])
        output_lang.add_sentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        # Как помним hidden_size - размер скрытого состояния
        self.hidden_size = hidden_size
        # Слой Эмбеддингов, который из входного вектора последовательности (либо батча) отдаст представление последовательности для скрытого состояния
        # FYI: в качестве Input_size у нас размер словаря
        self.embedding = nn.Embedding(input_size, hidden_size)
        # И соответственно рекуррентная ячейка GRU которая принимает MxM (hidden на hidden)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        # Приводим эмбеддинг к формату одного предлоежния 1х1 и любая размерность
        embedded = self.embedding(input).view(1, 1, -1)
        # Нужно для следующего шага пока не запутываемся :) просто присвоили наш эмбеддинг
        output = embedded
        # и соответственно подаем все в ГРЮ ячейку (эмбеддинг и скрытые состояния)
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        # Дополнительно сделаем инициализацию скрытого представления (просто заполним нулями)
        return torch.zeros(1, 1, self.hidden_size, device=device)


class DecoderRNN(nn.Module):
    # Будьте внимательны, теперь на вход мы получаем размер скрытого представления
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        # Слой эмбеддингов - рамер словаря, размер скрытого представления
        self.embedding = nn.Embedding(output_size, hidden_size)
        # GRU скрытое состояние на скрытое
        self.gru = nn.GRU(hidden_size, hidden_size)
        # Переводим hidden size в распределение для этого передаем в линейный слов скрытое состояни и размер словаря
        self.out = nn.Linear(hidden_size, output_size)
        # Получаем распределение верояностей
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0])) # берем output по нулевому индексу (одно предложение)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)


# Токены кодируем в целочисленное представление
def indexesFromSentence(lang, sentence):
    #print(sentence)
    return [lang.word2index[word] for word in sentence.split(' ')]


# Берем предложение с указанным языком, делаем из него индексы и вставляем метку конца предложения, превращаем в тензор:
def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    #print('indexes = ',indexes)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

# Для создания тензора из пар:
def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    #print('input_tensor = ', input_tensor)
    #print('tensorsFromPair shape')
    #print(input_tensor.shape)
    #print(target_tensor.shape)
    return (input_tensor, target_tensor)


teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    # Просто инициализируем скрытое представление для энкодера
    encoder_hidden = encoder.initHidden()
    # Скиыдваем градиенты для алгоритма градиентного спуска как и у энкодера так и у дэкодера
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    # Получаем размер в словаря (токенов) для входящего и выходящего тензора так как мы пробегаемся по каждому предложению по кусочкам
    #print('input_tensor.shape = ', input_tensor.shape)
    input_length = input_tensor.size(0)
    #print('target_length = ',target_tensor.size(0))
    target_length = target_tensor.size(0)
    max_length = input_length
    #print('input_length = {}  target_length = {} '.format(input_length,target_length))
    # Создаем переменную где будем хранить наши выходы из энкодера (в данной реализации пока не юзаем, далее будет еще один вариант)
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
    loss = 0
    # пробегаем по длине входящего тензора и в экодер передаем последовательно каждый из токенов:
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        # Сохраняем все выходы из энкодера для одного слова
        #print('encoder_output = ',encoder_output,encoder_output[0, 0])
        encoder_outputs[ei] = encoder_output[0, 0]


    # Закончили с энкодером пошли к декодеру, как было сказано декодер начинается с SOS
    decoder_input = torch.tensor([[SOS_token]], device=device)
    # FYI здесь мы скрытое представление из энкодера передаем в скрытое представление в декодер, то есть после знака =
    # у нас будут ходить градиенты из декодера в энкодер, то есть когда мы будем считать градиенты, они сначала пробегут по декодеру
    # дойдут до знака = перескочат в энкодер и будут дальше считаться по энкодеру и эти градиенты сохранятся в соответствующих тензорах
    # и когда будут отрабатывать разные оптимайзеры (у нас их 2) у них будут соответствующие правильные градиенты которые смогут правильно отработать
    decoder_hidden = encoder_hidden

    # Будем использовать Teacher Forcing в части случае (подставляя правильную последовательность)
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    if use_teacher_forcing:
        train_acc_sum = 0
        n_train = 0

        #train_acc_sum_top = 0
        #n_train_top = 0

        # Подаем decoder_input = torch.tensor([[SOS_token]], device=device) то есть по одному слову и скрытое представление
        for di in range(target_length):
            # Переведенное предложение и скрытое представление
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            #print('decoder', decoder_output)
            #softmaxed = torch.softmax(decoder_output, 1) 
            #probas = torch.distributions.multinomial.Multinomial(1, softmaxed).sample()  # нормализация
            #probas.max(dim=1)[1]
            #print('probas.argmax = ',probas.argmax(dim=1), target_tensor[di])
            #train_acc_sum += (probas.max(dim=1)[1] == target_tensor[di].item()).sum().item()
            #n_train += target_tensor[di].shape[0]
            # Считаем ошибку
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing
            
            topv, topi = decoder_output.data.topk(1)
            #print('topi = ',topi,target_tensor[di].item())
            train_acc_sum += (topi == target_tensor[di].item()).sum().item()
            n_train += target_tensor[di].shape[0]  
        #print('train_acc_sum_top = {}  n_train_top = {}    Train_acc_top: {:.3f}'.format(train_acc_sum_top, n_train_top, train_acc_sum_top / n_train_top))
    else:
        train_acc_sum = 0
        n_train = 0
        #train_acc_sum_top = 0
        #n_train_top = 0

        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input
            
            #сначало смотрел acc таким образом
            #softmaxed = torch.softmax(decoder_output, 1) 
            #probas = torch.distributions.multinomial.Multinomial(1, softmaxed).sample()  # нормализация
            #train_acc_sum += (probas.max(dim=1)[1] == target_tensor[di].item()).sum().item()
            #n_train += target_tensor[di].shape[0]
            
            loss += criterion(decoder_output, target_tensor[di])

            #смотрим acc
            topv, topi = decoder_output.data.topk(1)
            #print('topi = ',topi,target_tensor[di].item())
            train_acc_sum += (topi == target_tensor[di].item()).sum().item()
            n_train += target_tensor[di].shape[0]  

            if decoder_input.item() == EOS_token:
                break
        #print('train_acc_sum_top = {}  n_train_top = {}    Train_acc_top: {:.3f}'.format(train_acc_sum_top, n_train_top, train_acc_sum_top / n_train_top))
    loss.backward()
    encoder_optimizer.step()
    decoder_optimizer.step()
    return loss.item() / target_length,  train_acc_sum / n_train


import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / percent
    rs = es - s
    return '%s (- eta: %s)' % (asMinutes(s), asMinutes(rs))


import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
plt.switch_backend('agg')


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)


def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    # Делаем выборку наших пар функцией которую создали до
    training_pairs = [tensorsFromPair(random.choice(pairs)) for i in range(n_iters)]#[ tensorsFromPair(['tom was wearing an orange jumpsuit and his hands were cuffed in front of him .',
                      #                  'том был одет в оранжевыи комбинезон и его руки были скованы спереди .'])] 
                                        # [tensorsFromPair(random.choice(pairs)) for i in range(n_iters)]
    #print('training_pairs = ',training_pairs)
    #print('training_pairs ', training_pairs[0])
    # FYI! Используем Negative Log-Likelihood Loss потому что log softmax уже присутствует в модели
    criterion = nn.NLLLoss()

    for epoch in range(1, n_iters + 1):
        training_pair = training_pairs[epoch - 1]
        #print('training_pair = ',training_pair)
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]
        # Используем функцию для тренировки на отдельных токенах, которую написали выше
        #print('input_tensor',input_tensor.shape)
        loss, train_acc = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f  train_acc = %.3f' % (timeSince(start, epoch / n_iters),
                                         epoch, epoch / n_iters * 100, print_loss_avg, train_acc))

        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0
    showPlot(plot_losses)


def evaluate(encoder, decoder, sentence, out_sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        output_tensor = tensorFromSentence(output_lang, out_sentence)
        #print('input_tensor = ',input_tensor.shape, output_tensor.shape)
        input_length = input_tensor.size()[0]
        max_length = input_length
        encoder_hidden = encoder.initHidden()
        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for i in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[i], encoder_hidden)
            encoder_outputs[i] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS
        decoder_hidden = encoder_hidden
        decoded_words = [] # Наши деокдированные слова

        acc_sum = 0
        n = 0

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            #print('topi.item() = ',topi.item())
            #softmaxed = torch.softmax(decoder_output, 1) 
            #probas = torch.distributions.multinomial.Multinomial(1, softmaxed).sample()  # нормализация
            #probas.max(dim=1)[1]
            #print('probas.argmax = ',topi, output_tensor[di].item())
            acc_sum += (topi == output_tensor[di].item()).sum().item()
            #print(output_tensor[di].shape[0])
            n += output_tensor[di].shape[0]            
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])
            decoder_input = topi.squeeze().detach()
        #print(decoded_words)
        #print('Learning inference Acc: {:.3f}'.format(train_acc_sum / n_train))
        return decoded_words, acc_sum / n


def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, acc = evaluate(encoder, decoder, pair[0], pair[1])
        output_sentence = ' '.join(output_words)
        print('< {}   Learning inference Acc: {:.3f}'.format(output_sentence, acc))
        print('')

dataset = CreateDataset()
path = '/home/mikhail/it-academy/модуль 5/lecture_7_deepRNN/data/translation_data/rus.txt'
df = dataset.read_csv(path)   
#df = df[:1]
#print(df)
input_lang, output_lang, pairs = prepare_data('eng', 'rus', False)
#print('pairs = ',pairs, input_lang.n_words, output_lang.n_words)

hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder1 = DecoderRNN(hidden_size, output_lang.n_words).to(device)
#print(encoder1)
#print(decoder1)
trainIters(encoder1, decoder1, 450000, print_every=45000) #75000 число итераций

print()
print()
evaluateRandomly(encoder1, decoder1)

Reading lines...
Read 451436 sentence pairs
Trimmed to 451436 sentence pairs
Counting words...
Counted words:
eng 16741
rus 56729
0m 11s (- eta: 1m 45s) (100 10%) 6.3424  train_acc = 0.250
0m 24s (- eta: 1m 37s) (200 20%) 5.8945  train_acc = 0.667
0m 36s (- eta: 1m 25s) (300 30%) 5.7738  train_acc = 0.400
0m 49s (- eta: 1m 14s) (400 40%) 5.8043  train_acc = 0.286
1m 2s (- eta: 1m 2s) (500 50%) 5.7113  train_acc = 0.000
1m 15s (- eta: 0m 50s) (600 60%) 5.6519  train_acc = 0.167
1m 29s (- eta: 0m 38s) (700 70%) 5.5530  train_acc = 0.000
1m 42s (- eta: 0m 25s) (800 80%) 5.5163  train_acc = 0.000
1m 55s (- eta: 0m 12s) (900 90%) 5.5093  train_acc = 0.000
2m 8s (- eta: 0m 0s) (1000 100%) 5.2993  train_acc = 0.200


> i just got promoted .
= меня только что повысили .
< том не не не . <EOS>   Learning inference Acc: 0.333

> i think it s time for me to write my mother another letter .
= думаю пора мне написать матери еще одно письмо .
< том не не не . <EOS>   Learning inference Acc: 0.000

>

In [57]:
def evaluate(encoder, decoder, sentence, out_sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        output_tensor = tensorFromSentence(output_lang, out_sentence)
        #print('input_tensor = ',input_tensor)
        input_length = input_tensor.size()[0]
        max_length = input_length
        encoder_hidden = encoder.initHidden()
        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for i in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[i], encoder_hidden)
            encoder_outputs[i] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS
        decoder_hidden = encoder_hidden
        decoded_words = [] # Наши деокдированные слова

        acc_sum = 0
        n = 0

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            #print('topi.item() = ',topi.item())
            #softmaxed = torch.softmax(decoder_output, 1) 
            #probas = torch.distributions.multinomial.Multinomial(1, softmaxed).sample()  # нормализация
            #probas.max(dim=1)[1]
            #print('probas.argmax = ',topi, output_tensor[di].item())
            acc_sum += (topi == output_tensor[di].item()).sum().item()
            n += input_tensor[di].shape[0]            
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])
            decoder_input = topi.squeeze().detach()
        #print(decoded_words)
        #print('Learning inference Acc: {:.3f}'.format(train_acc_sum / n_train))
        return decoded_words, acc_sum / n

def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, acc = evaluate(encoder, decoder, pair[0], pair[1])
        output_sentence = ' '.join(output_words)
        print('< {}   Learning inference Acc: {:.3f}'.format(output_sentence, acc))
        print('')

evaluateRandomly(encoder1, decoder1)

> the new park is due to be open in october .
= новыи парк должен быть открыт в октябре .
< в парк в в парк . <EOS>   Learning inference Acc: 0.143

> i fell in love with someone my parents didn t like .
= я влюбилась в человека которыи не нравится моим родителям .
< я не не в в не не . <EOS>   Learning inference Acc: 0.222

> things didn t turn out the way tom expected .
= все вышло не так как том ожидал .
< не не не том не как <EOS>   Learning inference Acc: 0.143

> i m afraid it s going to rain tomorrow .
= боюсь что завтра будет дождь .
< боюсь завтра дождь дождь . <EOS>   Learning inference Acc: 0.167

> do you really want to marry me ?
= вы деиствительно хотите на мне жениться ?
< вы деиствительно хотите на меня ? <EOS>   Learning inference Acc: 0.571

> it was a competition .
= это было соревнование .
< это было ошибкои . <EOS>   Learning inference Acc: 0.800

> i haven t read many books .
= я не так много книг читал .
< я не читал книг книг . <EOS>   Learning inference Acc: 0.

In [39]:
output_lang.index2word[1998]

'ахнул'